In [1]:
#%load_ext profila

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os, subprocess, sys
import tqdm
from pathlib import Path
import glob
import joblib
import time
import numba

sys.path.append(str(Path(os.getcwd()).parent))

In [3]:
from src import extract_tree_stats, curve_inference, estimators, loader, tests, abc

# Speedups

In [4]:
def testme():
    # Run 
    start_time=time.time()
    abc.generate()
    end_time = time.time()
    diff = end_time - start_time
    print(f"Execution and compile time: {diff} seconds")
    
    start_time=time.time()
    abc.generate()
    end_time = time.time()
    print(f"Execution time: {end_time - start_time} seconds")
    


## Before any changes

In [17]:
testme()

Execution and compile time: 0.0006232261657714844 seconds
Execution time: 0.0004684925079345703 seconds


## Adding external loop of 100 iterations

Pretty much as expected, x100 times 

In [22]:
start_time=time.time()
for n_reps in range(100):
    abc.generate()
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Execution time: 0.03728342056274414 seconds


## Moving the loop internally

In [6]:
testme()

/rds/project/rds-8b3VcZwY7rY/users/rs2145/git/dated_selection/src/abc.py:131: NumbaDebugInfoWarning: Could not find source for function: <function jit_wrapper__built_in_method_random_of_numpy_random_mtrand_RandomState_object_at_0x1549d34f2d40_ at 0x1549b2a98dc0>. Debug line information may be inaccurate.
  somethinghappened = np.random.exponential(1 / lambda_alpha) if lambda_alpha > 0 else time_to_next_event
/rds/project/rds-8b3VcZwY7rY/users/rs2145/git/dated_selection/src/abc.py:131: NumbaDebugInfoWarning: Could not find source for function: <function jit_wrapper__built_in_method_exponential_of_numpy_random_mtrand_RandomState_object_at_0x1549d34f2d40_ at 0x1549b2fdc040>. Debug line information may be inaccurate.
  somethinghappened = np.random.exponential(1 / lambda_alpha) if lambda_alpha > 0 else time_to_next_event
/rds/project/rds-8b3VcZwY7rY/users/rs2145/git/dated_selection/src/abc.py:132: NumbaDebugInfoWarning: Could not find source for function: <function jit_wrapper__built_in_fu

Execution and compile time: 11.476057052612305 seconds
Execution time: 0.03850603103637695 seconds


In [27]:
print("Saved around ", (1-0.02596/0.037)*100, "percent of running time")

Saved around  29.83783783783783 percent of running time


## Profiling with `profila` to see where's the bottleneck

In [10]:
testme()

Execution and compile time: 0.03810596466064453 seconds
Execution time: 0.038697242736816406 seconds


In [ ]:
%%profila
for i in range(10):
    abc.generate()

There doesn't seem to be a strong bottleneck. Removing the strings might help a bit, but probably not dramatically.

## Does fastmath help?

Without

In [17]:
testme()

Execution and compile time: 0.03928112983703613 seconds
Execution time: 0.038401126861572266 seconds


With

In [11]:
testme()

Execution and compile time: 0.025449275970458984 seconds
Execution time: 0.025081634521484375 seconds


Yes it seems to take some ~30% out

## What about parallel=True?

In [ ]:
testme()